# Identificação de Cartas Ideais em um Deck com Jirachi como Alvo

Esse notebook analisa as cartas ideais em um deck que contém a carta `Jirachi` como alvo.

### Bibliotecas utilizadas e carregando dataset

In [1]:
import numpy as np
import pandas as pd
import pysubgroup as ps


df_original = pd.read_csv('./Data/tournaments.csv')
df = df_original.copy()

In [2]:
POKEMON_ALVO = 'Jirachi'
CONSIDERAR_TREINADORES = True
ANO_ANALISAR = 2019
 
CARTAS_MINIMAS = 20

### Eliminar Cartas de Treinador (se necessário)

In [3]:
if not CONSIDERAR_TREINADORES:
    df = df[df['type_card'] != 'Trainer']

df.head()

,id_card,name_card,amount_card,price_card,energy_type_card,type_card,combo_type_id,combo_type_name,id_player,name_player,...,region_tournament,country_tournament,year_tournament,month_tournament,day_tournament,valid_rotation_at_tournament,rotation_name,year_begin,month_begin,day_begin
0,SSP272,Archeops,4,0.70,Colorless,Pokémon,247.0,Lugia Archeops,5101,Alexander Flatos,...,NaN,United States,2023,10,14,BST-OBF,standard_2023,2023,4,14
1,SIT138,Lugia V,3,6.20,Colorless,Pokémon,247.0,Lugia Archeops,5101,Alexander Flatos,...,NaN,United States,2023,10,14,BST-OBF,standard_2023,2023,4,14
2,SIT139,Lugia VSTAR,3,8.71,Colorless,Pokémon,247.0,Lugia Archeops,5101,Alexander Flatos,...,NaN,United States,2023,10,14,BST-OBF,standard_2023,2023,4,14
3,LOR143,Snorlax,3,0.93,Colorless,Pokémon,247.0,Lugia Archeops,5101,Alexander Flatos,...,NaN,United States,2023,10,14,BST-OBF,standard_2023,2023,4,14
4,SSP250,Lumineon V,2,1.90,Water,Pokémon,247.0,Lugia Archeops,5101,Alexander Flatos,...,NaN,United States,2023,10,14,BST-OBF,standard_2023,2023,4,14


### Filtragem de Colunas no Dataset e Limpeza de Linhas com Coluna Região do Torneio Vazias

In [4]:
columns_to_remove = [
    'id_card', 'amount_card', 'price_card', 'combo_type_id', 'name_tournament',
    'combo_type_name', 'name_player', 'category_tournament', 'type_card',
    'month_tournament', 'day_tournament', 'valid_rotation_at_tournament', 
    'rotation_name', 'year_begin', 'all_time_score', 'country_tournament', 'country_player',
    'month_begin', 'day_begin',
]
df.drop(columns=columns_to_remove, inplace=True)
df = df[df['region_tournament'].notna()]

df.head(5)

,name_card,energy_type_card,id_player,ranking_player_tournament,id_tournament,region_tournament,year_tournament
7462,Charmander,Fire,649,1,385,SA,2023
7463,Charmeleon,Fire,649,1,385,SA,2023
7464,Charizard ex,Darkness,649,1,385,SA,2023
7465,Pidgey,Colorless,649,1,385,SA,2023
7466,Pidgeot ex,Colorless,649,1,385,SA,2023


### Separando por Região para Verificar Recomendações de Cartas para o Deck com `Jirachi`

In [5]:
def elimina_cards_menos_usados(df):
    cartas = df['name_card'].unique()
    for carta in cartas:
        if len(df[df['name_card'] == carta]) < CARTAS_MINIMAS:
            df = df[df['name_card'] != carta]
    return df

def agrupa_deck(df_region):
    #filtra torneios da regiao
    torneios = df_region['id_tournament'].unique()

    lista_decks = []
    for torneio in torneios:
        df_torneio = df_region[df_region['id_tournament'] == torneio]
        jogadores = df_torneio['id_player'].unique()
        for jogador in jogadores:
            df_jogador = df_torneio[df_torneio['id_player'] == jogador]
            cartas_jogador = df_jogador['name_card'].unique().tolist()
            lista_decks.append(cartas_jogador)
    return lista_decks

def subgrupo_dataset_region(df_region):
    lista_decks = agrupa_deck(df_region)
    cartas = df_region['name_card'].unique()
    data = []
    for deck_rank in lista_decks:
        pokemons_no_deck = {carta: (carta in deck_rank[:len(deck_rank)-1]) for carta in cartas}
        data.append(pokemons_no_deck)
    # Converte a lista de dicionários em um DataFrame
    df_decks_cartas = pd.DataFrame(data)

    return df_decks_cartas

def calcula_subgrupo(df):
    target = ps.BinaryTarget(POKEMON_ALVO, True)
    search_space = ps.create_selectors(df, ignore=[POKEMON_ALVO])
    task = ps.SubgroupDiscoveryTask(
        df,
        target, 
        search_space, 
        result_set_size=20, 
        depth=5, 
        qf=ps.WRAccQF()  # Quality function to evaluate subgroups
    )
    result = ps.BeamSearch().execute(task)
    return result

### Por regiao

In [6]:
regions = df['region_tournament'].unique()
for region in regions:
    print(f"Região: {region}\n")
    df_region = df[(df['region_tournament'] == region) & (df['year_tournament'] == ANO_ANALISAR)]

    # Verifica se há torneios na região
    if len(df_region) == 0:
        print(f'Sem torneios na região {region} no ano {ANO_ANALISAR}\n')
        print("-------------------------------------------"*3)
        continue

    # Verifica se há a carta POKEMON_ALVO na região
    if POKEMON_ALVO not in df_region['name_card'].unique():
        print(f'Sem a carta {POKEMON_ALVO} na região {region}\n')
        print("-------------------------------------------"*3)
        continue

    df_region = elimina_cards_menos_usados(df_region)
    dataset = subgrupo_dataset_region(df_region)
    result = calcula_subgrupo(dataset)

    for row in result.to_dataframe().itertuples():
        print(f"Subgrupo: {row.subgroup}")
    print()
    print(result.to_dataframe(), "\n")
    print("-------------------------------------------"*3)


Região: SA

Subgrupo: Pokégear 3.0==False AND Switch==True
Subgrupo: Chaotic Swell==False AND Pokégear 3.0==False AND Switch==True
Subgrupo: Custom Catcher==False AND Pokégear 3.0==False AND Switch==True
Subgrupo: Chaotic Swell==False AND Custom Catcher==False AND Pokégear 3.0==False AND Switch==True
Subgrupo: Custom Catcher==False AND Switch==True
Subgrupo: Chaotic Swell==False AND Custom Catcher==False AND Switch==True
Subgrupo: Pokégear 3.0==False AND Reset Stamp==True AND Switch==True
Subgrupo: Great Catcher==True AND Pokégear 3.0==False AND Switch==True
Subgrupo: Chaotic Swell==False AND Pokégear 3.0==False AND Reset Stamp==True AND Switch==True
Subgrupo: Chaotic Swell==False AND Great Catcher==True AND Pokégear 3.0==False AND Switch==True
Subgrupo: Acro Bike==False AND Custom Catcher==False AND Switch==True
Subgrupo: Acro Bike==False AND Chaotic Swell==False AND Custom Catcher==False AND Switch==True
Subgrupo: Acro Bike==False AND Pokégear 3.0==False AND Switch==True
Subgrupo: Ac

### Análise Ignorando Regiões

In [7]:
df_all = df[(df['year_tournament'] == ANO_ANALISAR)]

# verifica se tem torneios
if len(df_all) == 0:
    print(f'Sem torneios no ano {ANO_ANALISAR}')
    print("-------------------------------------------"*3)
    exit()

# verifica se tem a carta POKEMON_ALVO
if POKEMON_ALVO not in df_all['name_card'].unique():
    print(f'Sem a carta {POKEMON_ALVO} no ano {ANO_ANALISAR}')
    print("-------------------------------------------"*3)
    exit()

# calcula subgrupo
df_all = elimina_cards_menos_usados(df_all)
dataset = subgrupo_dataset_region(df_all)
result = calcula_subgrupo(dataset)

# imprime resultados
for row in result.to_dataframe().itertuples():
    print(f"Subgrupo: {row.subgroup}")
print()
print(result.to_dataframe(), "\n")
print("-------------------------------------------"*3)

Subgrupo: Escape Board==True AND Green's Exploration==False
Subgrupo: Escape Board==True AND Green's Exploration==False AND Zorua==False
Subgrupo: Escape Board==True AND Green's Exploration==False AND Zoroark-GX==False
Subgrupo: Escape Board==True AND Green's Exploration==False AND Zoroark-GX==False AND Zorua==False
Subgrupo: Escape Board==True AND Green's Exploration==False AND Multi Switch==False
Subgrupo: Escape Board==True AND Green's Exploration==False AND Multi Switch==False AND Zorua==False
Subgrupo: Escape Board==True AND Green's Exploration==False AND Multi Switch==False AND Zoroark-GX==False
Subgrupo: Escape Board==True AND Green's Exploration==False AND Multi Switch==False AND Zoroark-GX==False AND Zorua==False
Subgrupo: Chaotic Swell==False AND Escape Board==True AND Green's Exploration==False
Subgrupo: Chaotic Swell==False AND Escape Board==True AND Green's Exploration==False AND Zorua==False
Subgrupo: Chaotic Swell==False AND Escape Board==True AND Green's Exploration==Fa